# Classification d'image vue de face de veaux avec Unet

## Import des dependances

In [1]:
from sklearn.model_selection import train_test_split
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
from torch.utils.data import Dataset
from transformers import Trainer, TrainingArguments
import torch
import torch.nn as nn
import torchvision.models.segmentation as segmentation
from torchvision import models
from torchvision.transforms import v2
from torchvision.io import read_image
import numpy as np
import cv2
import os
import sys
import torch.optim as optim
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score, recall_score, f1_score, classification_report
from tqdm import tqdm
import random
from PIL import Image as PilImage
from omnixai.data.image import Image
from omnixai.explainers.vision import LimeImage, VisionExplainer
from omnixai.preprocessing.image import Resize
from efficientnet_pytorch import EfficientNet
from torchsampler import ImbalancedDatasetSampler
from IPython.display import clear_output
import timm

# Local dep
project_dir = '/data/konrad/workspace'
sys.path.insert(0, project_dir)

from helpers.helpers import get_indices, load_face_data, load_local_model
from helpers.datasets import CalfCenterFaceDataset
from helpers.interp import GradCam, generate_cam, display_predicted_cam
from helpers.trainers import train_model, validate_model, plot_metrics

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

2024-08-27 13:55:33.276719: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-27 13:55:33.287113: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-27 13:55:33.290293: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-27 13:55:33.299170: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-27 13:55:33.990973: W tensorflow/compiler/tf2

## Classes et utilitaires

In [2]:
class Classifier(nn.Module):
    def __init__(self, num_classes, name, from_pretrained=True):
        super(Classifier, self).__init__()
        self.name = name
        self.unet = segmentation.deeplabv3_resnet50(pretrained=from_pretrained)  # Load the U-Net backbone

        # Remove the last segmentation layer
        self.unet.classifier = nn.Identity()

        self.conv_head = nn.Conv2d(2048, num_classes, kernel_size=(1, 1), stride=(1, 1), bias=False)
        self.classifier_head = nn.Sequential(
            nn.AdaptiveAvgPool2d((1, 1)),  # Global average pooling
            nn.Flatten(),
        )

    def forward(self, x):
        features = self.unet(x)['out']
        features = self.conv_head(features)
        output = self.classifier_head(features)
        return output

## Entrainement du modele

### Dataset setup

In [3]:
# train_indices, test_indices, valid_indices = get_indices(len(df))
ROOT_DIR = "/data/konrad/workspace"
DATA_FILE = ROOT_DIR + "/csv_files/CompleteDatasetNormalFace/Face_annotations.csv"
IMAGE_DIR = ROOT_DIR + "/csv_files/CompleteDatasetNormalFace"
IMAGE_SIZE = 512

# dataset_type = "splt"
dataset_type = "loco"

# df, labels, label2id, id2label = load_face_data(DATA_FILE, IMAGE_DIR)
# root_dir = ROOT_DIR + f"/datasets/mixed_{frame}s_b{step}s"
# root_dir = f"/data/data_calves/konrad/mixed_{frame}s_b{step}s"

train_df = pd.read_csv(ROOT_DIR + f'/csv_files/mixed_10s_b0s_y7_1/{dataset_type}_train_image_extracted_metadata.csv', index_col=False)
valid_df = pd.read_csv(ROOT_DIR + f'/csv_files/mixed_10s_b0s_y7_1/{dataset_type}_test_image_extracted_metadata.csv', index_col=False)
test_df = pd.read_csv(ROOT_DIR + f'/csv_files/mixed_10s_b0s_y7_1/{dataset_type}_val_image_extracted_metadata.csv', index_col=False)

label_col = "bilabel"
# label_col = "label"
# train_df = train_df[~ (train_df[label_col] == "Diarrhé, Pneumonie")]
# valid_df = valid_df[~ (valid_df[label_col] == "Diarrhé, Pneumonie")]
# test_df = test_df[~ (test_df[label_col] == "Diarrhé, Pneumonie")]

labels = train_df[label_col].unique()
label2id = {l:i for i, l in enumerate(labels)}
id2label = {i:l for i, l in enumerate(labels)}

# num_labels = len(labels)
num_labels = 1
train_df['target'] = train_df.apply(lambda row: label2id[row[label_col]], axis=1)
test_df['target'] = test_df.apply(lambda row: label2id[row[label_col]], axis=1)
valid_df['target'] = valid_df.apply(lambda row: label2id[row[label_col]], axis=1)
test_df = test_df.sample(frac=.10).reset_index(drop=True)

train_transform = v2.Compose([
    v2.Resize(size=(IMAGE_SIZE, IMAGE_SIZE)),
    v2.RandomHorizontalFlip(p=0.5),
    v2.AutoAugment(v2.AutoAugmentPolicy.IMAGENET),
    v2.ToDtype(torch.float32, scale=True),
    v2.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

test_transform = v2.Compose([
    v2.Resize(size=(IMAGE_SIZE, IMAGE_SIZE)),
    v2.ToDtype(torch.float32, scale=True),
    v2.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

train_dataset = CalfCenterFaceDataset(train_df, transform=train_transform)
test_dataset = CalfCenterFaceDataset(test_df, transform=test_transform)
valid_dataset = CalfCenterFaceDataset(valid_df, transform=test_transform)

# Data loaders
batch_size = 2
train_loader = DataLoader(train_dataset, batch_size=batch_size, sampler=ImbalancedDatasetSampler(train_dataset))
valid_loader = DataLoader(valid_dataset, batch_size=batch_size * 2, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size * 2, shuffle=False)

# dl = valid_loader.dataset.data_frame
# class_counts = dl['label'].value_counts()

# # Plot the distribution
# plt.figure(figsize=(8, 6))
# class_counts.plot(kind='bar')
# plt.title('Distribution of Classes in Validation set')
# plt.xlabel('Class')
# plt.ylabel('Count')
# plt.xticks(rotation=45)
# plt.grid(axis='y', linestyle='--', alpha=0.7)
# plt.show()

### Entrainement et validation

In [4]:
model_name = f"efficientnet_{dataset_type}_{label_col}_{batch_size}"
model = Classifier(num_classes=num_labels, name=model_name, from_pretrained=True)  # Adjust num_classes according to your dataset
model.to(device)

# Loss and optimizer
lr = 1e-4
optimizer = optim.Adam(model.parameters(), lr=lr)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=2, verbose=True)

epochs = 15
patience = 3

if num_labels == 1:
    average = "binary"
else:
    average = "weighted"

output_dir = ROOT_DIR + "/training_log"
metrics_scores = train_model(model, labels, train_loader, test_loader, scheduler, patience, optimizer, output_dir, num_epochs = epochs, average=average)
clear_output()
plot_metrics(metrics_scores)

/data/konrad/x_env/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/data/konrad/x_env/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DeepLabV3_ResNet50_Weights.COCO_WITH_VOC_LABELS_V1`. You can also use `weights=DeepLabV3_ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Training on 213 and validating on 694 datas


100%|█████████████████████████████████████████████████████████████████████████████████| 107/107 [00:33<00:00,  3.18it/s]


Epoch [1/15]
Train Loss: 0.0067, Accuracy: 0.5211, Recall: 0.7553, F1 Score: 0.5820
Validating on 694 datas


100%|█████████████████████████████████████████████████████████████████████████████████| 174/174 [01:05<00:00,  2.67it/s]


Test Accuracy: 0.3141, Recall: 0.9070, F1 Score: 0.3959, Bal Acc Score: 0.5129
------------------------------


 47%|██████████████████████████████████████▎                                           | 50/107 [00:14<00:16,  3.37it/s]


KeyboardInterrupt: 

In [ ]:
model.load_state_dict(torch.load(f'{output_dir}/models/best_{model.name}_model.pth'))
validate_model(model, valid_loader, labels, average=average)

## Interpretation with Lime

In [ ]:
# model = Classifier(num_classes=len(labels), name=model_name, from_pretrained=False)  # Adjust num_classes according to your dataset
# model.load_state_dict(torch.load(f'{output_dir}/models/best_{model.name}_model.pth'))
# model = model.to(device)

In [ ]:
# test_df = valid_loader.dataset.data_frame.copy(deep=True)
# test_df.reset_index(drop=True, inplace=True)
# test_images = [Resize((256, 256)).transform(Image(PilImage.open(p["path"]).convert('RGB'))).to_numpy() for p in test_df.to_dict('records')]
# img = Image( data=np.concatenate(test_images), batched=True)

# # The preprocessing function
# transform = v2.Compose([
#     v2.Resize(256),
#     v2.CenterCrop(224),
#     v2.ToImage(),
#     v2.ToDtype(torch.float32, scale=True),
#     v2.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
# ])
# preprocess = lambda ims: torch.stack([transform(im.to_pil()) for im in ims]).to(device)
# postprocess = lambda logits: torch.nn.functional.softmax(logits, dim=1)

# target_head = model.conv_head
# target_layer = model.conv_head
# explainer = VisionExplainer(
#     explainers=[
#        "gradcam",
#         "lime", 
#         # "ig",
#         # "ce",
#         # "scorecam",
#         # "smoothgrad", 
#         "guidedbp", 
#         "layercam"
#     ],
#     mode="classification",
#     model=model,
#     preprocess=preprocess,
#     postprocess=postprocess,
#     params={
#         "gradcam": {"target_layer": target_head},
#         # "ce": {"binary_search_steps": 2, "num_iterations": 100},
#         # "scorecam": {"target_layer": target_head},
#         "layercam": {"target_layer": target_layer},
#     }
# )

# # Generate explanations
# local_explanations = explainer.explain(img)
# clear_output()

In [ ]:
# df_per_class = {l:test_df[test_df["label"] == l] for l in labels}
# class_id = 0
# l, f_df  = list(df_per_class.items())[class_id]
# print(f"Class label: {l}:\n\t")

# for row_id in f_df.index.values:

#     print(f"Interpretation pour image {row_id}: \n")
#     for name, explanations in local_explanations.items():
#         print(f"{name}:")
#         explanations.ipython_plot(row_id, class_names=labels)


# print("-" * 40)

In [ ]:
# df_per_class = {l:test_df[test_df["label"] == l] for l in labels}
# class_id = 1
# l, f_df  = list(df_per_class.items())[class_id]
# print(f"Class label: {l}:\n\t")

# for row_id in f_df.index.values:

#     print(f"Interpretation pour image {row_id}: \n")
#     for name, explanations in local_explanations.items():
#         print(f"{name}:")
#         explanations.ipython_plot(row_id, class_names=labels)


# print("-" * 40)

In [ ]:
# df_per_class = {l:test_df[test_df["label"] == l] for l in labels}
# class_id = 2
# l, f_df  = list(df_per_class.items())[class_id]
# print(f"Class label: {l}:\n\t")

# for row_id in f_df.index.values:

#     print(f"Interpretation pour image {row_id}: \n")
#     for name, explanations in local_explanations.items():
#         print(f"{name}:")
#         explanations.ipython_plot(row_id, class_names=labels)


# print("-" * 40)